In [1]:
from keplergl import KeplerGl 
import json
from pprint import pprint

 En este notebook vamos a explicar como representar los resultados de una query de REDATAM en un mapa.

Las librerías a utilizar para representar información sobre mapas son diversas. En este caso evaluaremos algunas opciones con el fin de decidir cuales mas apropiada para el trabajo

# Bajando información de REDATAM

Lo primero que vamos a hacer es bajar un dataframe para usar de ejemplo. Para esto harémos uso de las funciones en `redatam.py` que construimos anteriormente

In [4]:
from redatam import bajar_query, soup_to_dataframe, soup_to_dict

In [ ]:
query_total = 'HOGAR.H1510 = 2 OR HOGAR.H1510 = 1'
query_nbi_ii_arg = 'HOGAR.H1510 = 2'

soup_nbi_ii_arg = bajar_query(query_nbi_ii_arg)
data_nbi_ii_arg = soup_to_dict(soup_nbi_ii_arg)

soup_total = bajar_query(query_total)
data_total = soup_to_dict(soup_total)

# Archivos de radios censales

### Proyección

El instituto geográfico nacional brinda archivos shapefile con los polígonos de los radios censales. Laproyeccion de las coordenadas está en EPSG:3857 y hay que traducirlas a EPSG:4326.

Transformamos el archivo shape a geojson y lo convertimos a coordenadas EPSG:4326 utilizando una [herramienta web](https://mygeodata.cloud/converter/)

In [35]:
with open('../datos/Codgeo_Santa_Fe_con_datos/mygeodata/Santa_Fe_con_datos.geojson') as geojson:
    radios_santa_fe = json.load(geojson)

### Metadata

El archivo de radios censales descargado del IGA (Instituto Geográfico Nacional) tiene algunos datos sobre población total, hogares y población de hombres y mujere. En nuestro caso no queremos esta información ya que los datos demográficos serán descargados de REDATAM.
El primer paso será borrar esta información del archivo GEOJSON.

In [36]:
for feature in radios_santa_fe['features']:
    feature['properties'] = {'link': feature['properties']['link']}

Por otro lado, todos los radios censales son polígonos y, para algunas representaciones espaciales (como ser mapas de calor) necesitamos tener sólo un punto en lugar del radio censal completo. Para este fin utilizaremos el centroide del radio censal y lo añadirémos a la metadata de cada polígono.

El último paso será incorporar a la metadata de cada radio censal el resultado descargado de REDATAM

In [37]:
for feature in radios_santa_fe['features']:
    try:
        d_nbi_ii_arg = data_nbi_ii_arg[feature['properties']['link']]
        d_total = data_total[feature['properties']['link']]
        if not d_total['total']:
            continue
        metadata = {
            'total_hogares': d_total['total'],
            'hogares_nbi_ii': d_nbi_ii_arg['total'],
            'nbi_ii_%': round(d_nbi_ii_arg['total']*100 / d_total['total'], 2)
        }
    except KeyError:
        print("Radio censal no encontrado en REDATAM: {}".format(feature['properties']['link']))
    metadata['link'] = feature['properties']['link']
    feature['properties'] = metadata

Radio censal no encontrado en REDATAM: 821330631


In [38]:
with open('readios_censales_con_datos.json', 'w') as jsonfile:
    json.dump(radios_santa_fe, jsonfile)

# Kepler

Una de las librerías que me gustaría probar, por sus herramientas para representación de información geográfica, es [Kepler](https://kepler.gl/#/).

Encontré algunos recursos para utilizarla en conjunto con Jupyter notebook en [GitHub](https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md) y [Medium](https://medium.com/vis-gl/introducing-kepler-gl-for-jupyter-f72d41659fbf)



In [2]:
with open('readios_censales_con_datos.json') as jsonfile:
    radios_santa_fe = json.load(jsonfile)

with open('kepler.config.json') as jsonfile:
    kepler_config = json.load(jsonfile)

In [3]:
mapa_2 = KeplerGl(height=700, config=kepler_config)
mapa_2.add_data(radios_santa_fe, name="radios_censales")

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [4]:
mapa_2

KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': '67yqg3p', 'type': …